In [ ]:
base_dir = 'C:\\Projects\\ature'
import os
os.chdir(base_dir) #BASE DIRECTORY

### DRIVE dataset
data_file_path = 'data\\DRIVE\\test\\images'
mask_path = 'data\\DRIVE\\test\\mask'
ground_truth_path = 'data\\DRIVE\\test\\1st_manual'
mask_suffix =  '_test_mask.gif'
ground_truth_suffix = '_manual1.gif'

input_image = '19_test.tif'

In [ ]:
#!/home/akhanal1/Spring2018/pl-env/bin/python3.5
import sys
import numpy as np
import itertools as itr
from random import shuffle
from commons.segmentation import AtureTest
from PIL import Image as IMG
from commons.IMAGE import SegmentedImage
import cv2
import preprocess.utils.filter_utils as filutils
import preprocess.utils.img_utils as imgutils
import copy


SK_THRESHOLD_PARAMS = np.arange(40, 61, 20)
ALPHA_PARAMS = np.arange(5, 7, 0.5)
GABOR_CONTRIBUTION_PARAMS = np.arange(0.6, 1.1, 0.2)
SEGMENTATION_THRESHOLD_PARAMS = np.arange(9, 15, 0.5)

PARAMS_ITR = itr.product(SK_THRESHOLD_PARAMS, ALPHA_PARAMS, GABOR_CONTRIBUTION_PARAMS, SEGMENTATION_THRESHOLD_PARAMS)

PARAMS_COMBINATION = list(PARAMS_ITR)
shuffle(PARAMS_COMBINATION)

keys = ('sk_threshold', 'alpha', 'gabor_contrib', 'seg_threshold')

all_params = list(dict(zip(keys, param)) for param in PARAMS_COMBINATION)

params = {'sk_threshold': 60,
          'alpha': 9.0,
          'orig_contrib': 0.7,
          'seg_threshold': 20}

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def get_mask_file(file_name): return file_name.split('_')[0] + mask_suffix
def get_ground_truth_file(file_name): return file_name.split('_')[0] + ground_truth_suffix

img_obj = SegmentedImage()

img_obj.load_file(data_dir=data_file_path, file_name=input_image)
img_obj.load_mask(mask_dir=mask_path, fget_mask=get_mask_file, erode=True)
img_obj.load_ground_truth(gt_dir=ground_truth_path, fget_ground_truth=get_ground_truth_file)

img_obj.res['orig'] = img_obj.image_arr[:, :, 1]
img_obj.working_arr = img_obj.image_arr[:, :, 1]

img_obj.apply_mask()
img_obj.apply_bilateral()
img_obj.apply_gabor()

img_obj.apply_bilateral()
img_obj.working_arr = img_obj.res['bilateral']
# img_obj.apply_bilateral()
# img_obj.working_arr = img_obj.res['bilateral']

img_obj.generate_skeleton(threshold=params['sk_threshold'])
img_obj.generate_lattice_graph()

In [ ]:
### Run for image files with in-time mask erosion
tester = AtureTest(out_dir='out')
acc = tester.run(params=params, save_images=False, img_obj=img_obj)
# tester.run_all(params_combination=all_params)

In [ ]:
IMG.fromarray(img_obj.res['orig'])

In [ ]:
IMG.fromarray(img_obj.res['segmented'])

In [ ]:
print(img_obj.res['scores'])

In [ ]:
IMG.fromarray(img_obj.res['segmented_rgb'])

In [ ]:
fni = np.full(img_obj.res['segmented'].shape, 250, dtype=np.uint8)
for i in range(fni.shape[0]):
    for j in range(fni.shape[1]):
        if img_obj.res['segmented'][i,j] == 0 and img_obj.ground_truth[i,j] == 255:
                       fni[i, j] = img_obj.working_arr[i, j]

In [ ]:
m = img_obj.working_arr.copy()
for n1, n2 in img_obj.res['graph'].edges():
    m[n1] = 255
    m[n2] = 255
IMG.fromarray(m)

In [ ]:
IMG.fromarray(np.array(rgb, dtype=np.uint8))

In [ ]:
io = SegmentedImage()
io.working_arr = np.array(m.copy(),dtype=np.uint8)
io.res['orig'] = io.working_arr
io.generate_lattice_graph()
io.ground_truth = img_obj.ground_truth
io.mask = img_obj.mask

In [ ]:
io.generate_skeleton(threshold=135)
rgb = np.zeros([m.shape[0], m.shape[1], 3])
imgutils.rgb_scores(arr_2d=io.res['skeleton'], truth=io.ground_truth, arr_rgb=rgb)

In [ ]:
pms = {'sk_threshold': 60,
          'alpha': 9.0,
          'orig_contrib': 0.7,
          'seg_threshold': 40}
tester1 = AtureTest(out_dir='out')
acc = tester.run(params=pms, save_images=False, img_obj=io)

In [ ]:
IMG.fromarray(io.res['orig'])

In [ ]:
imgutils.histogram(fni, bins=10)

In [ ]:
ig = SegmentedImage()

In [ ]:
ig.working_arr = img_obj.working_arr[:,200:300]
ig.res['orig'] = img_obj.working_arr[:,200:300]
ig.res['skeleton'] = img_obj.res['skeleton'][:,200:300]

In [ ]:
IMG.fromarray(ig.working_arr)

In [ ]:
ig.ground_truth = img_obj.ground_truth[:,200:300]
ig.mask = img_obj.mask[:,200:300]
ig.apply_bilateral()
ig.working_arr = ig.res['bilateral']
ig.generate_lattice_graph()
ig.working_arr = ig.res['bilateral']
ig.generate_lattice_graph()

In [ ]:
pms = {'sk_threshold': 60,
          'alpha': 7.0,
          'orig_contrib': 0.2,
          'seg_threshold': 24}
tester1 = AtureTest(out_dir='out')
a = tester.run(params=pms, save_images=False, img_obj=ig)

In [ ]:
IMG.fromarray(ig.res['segmented_rgb'])

In [ ]:
ig.res['scores']

In [ ]:
cost = []
for n1,n2 in io.res['graph'].edges():
    cost.append(io.res['graph'][n1][n2]['cost'])

In [ ]:
sorted(cost, reverse=True)